# Slackbot Example

SlackBot keeps you in the loop without disturbing your focus. Its personalized, intelligent AI continuously monitors your Slack workspace, alerting you to important conversations and freeing you to concentrate on what’s most important.

SlackBot reads the full history of your (public) Slack workspace and trains a Generative AI model to predict when you need to engage with a conversation. This training process gives the AI a deep understanding of your interests, expertise, and relationships. Using this understanding, SlackBot watches conversations in real-time and notifies you when an important conversation is happening without you. With SlackBot200 you can focus on getting things done without worrying about missing out.

In this notebook, you’ll see you how to build and deploy SlackBot in 15 minutes using only OpenAI’s API’s and open-source Python libraries - Data Science PhD not required.



In [ ]:
%pip install openai kaskada

In [ ]:
from datetime import datetime, timedelta
from slack_sdk.socket_mode import SocketModeClient
from slack_sdk.socket_mode.response import SocketModeResponse
import sparrow_py as kt
import pandas
import openai
import getpass
import pyarrow
import datetime

# Initialize Kaskada with a local execution context.
kt.init_session()

# Initialize OpenAI
#openai.api_key = getpass.getpass('OpenAI: API Key')

## Fine-tune the model

### Read Historical Messages

***Issue***: `with_key` fails:

```
$ messages.with_key(kt.record({
        "channel": messages.col("channel"),
        "thread": messages.col("thread_ts"),
    }))

RuntimeError: error in sparrow-py Rust code
├╴at src/error.rs:54:21
│
├─▶ execute query
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-session/src/session.rs:319:14
│
├─▶ internal compute error: spawn compute executor
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute.rs:268:6
│
├─▶ internal compute error: unable to create operation
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute/operation.rs:435:6
│
├─▶ internal compute error: spead failure
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute/operation/merge.rs:171:106
│
╰─▶ Unsupported type Struct([Field { name: "count", data_type: Int64, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "name", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "users", data_type: List(Field { name: "item", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }), nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }]) for list spread
    ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute/operation/merge.rs:171:92
```

In [ ]:
messages = kt.sources.ArrowSource(
    data = pandas.read_parquet("./messages.parquet"), 
    time_column_name = "ts", 
    key_column_name = "channel",
)
#messages = messages.with_key(kt.record({  # !!!
#        "channel": messages.col("channel"),
#        "thread": messages.col("thread_ts"),
#    }))
messages.preview(5)

### Construct conversations

***Issue:*** Missing `seconds_since` 

***Issue:*** `collect` failes when given a `kt.SinceWindow` with

```
$ messages.select("user", "ts", "text", "reactions") \
    .collect(window=kt.SinceWindow(predicate=is_new_conversation), max=100)

ValueError: error[E0010]: Invalid argument type(s)
  --> Builder:1:1
  |
1 | builder
  | ^^^^^^^
  | |
  | Invalid types for parameter 'window' in call to 'collect'
  | Actual type: i64
  |
  --> built-in signature 'collect<T: any>(input: T, const max: i64, const min: i64 = 0, window: bool = null, duration: i64 = null) -> list<T>':1:71
  |
1 | collect<T: any>(input: T, const max: i64, const min: i64 = 0, window: bool = null, duration: i64 = null) -> list<T>
  |                                                                       ---- Expected type: bool


Arg[0]: Timestream[struct<ts: timestamp[ns], user: string, text: string, reactions: list<item: struct<count: int64, name: string, users: list<item: string>>>>]
Arg[1]: Literal 100 (<class 'int'>)
Arg[2]: Literal 0 (<class 'int'>)
Arg[3]: Literal True (<class 'bool'>)
Arg[4]: Literal None (<class 'NoneType'>)
```

In [ ]:
# Construct conversations
message_time = messages.time_of()
last_message_time = message_time.lag(1) # !!!
is_new_conversation = True #message_time.seconds_since(last_message_time) > 10 * 60

conversations = messages \
    .select("user", "ts", "text", "reactions") \
    .collect(max=100) #.collect(window=kt.SinceWindow(predicate=is_new_conversation), max=100)

conversations.preview(5)

### Build examples

***Issue***: Unable to field-ref into a list of records

```
$ conversations.col("reactions").col("name").preview(5)

RuntimeError: error in sparrow-py Rust code
├╴at src/error.rs:54:21
│
├─▶ execute query
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-session/src/session.rs:319:14
│
├─▶ internal compute error: spawn compute executor
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute.rs:268:6
│
├─▶ internal compute error: unable to create executor
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute/operation.rs:193:18
│
╰─▶ Field-ref only works on lists of records, but was List(Field { name: "item", data_type: List(Field { name: "item", data_type: Struct([Field { name: "count", data_type: Int64, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "name", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "users", data_type: List(Field { name: "item", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }), nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }]), nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }), nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} })
    ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute/operation.rs:192:18
```

***Issue***: Missing `union` function

***Issue***: Unable to collect a field from a record

```
$ conversations.col("user").collect(max=10).preview(5)

PanicException: not implemented: list collect evaluator is unsupported
```

***Issue***: Unable to combine shifted and unshifted values

```
$ kt.record({
    "prompt": conversations.shift_by(datetime.timedelta(minutes=5)),
    "completion": conversations,
}).preview(5)

RuntimeError: error in sparrow-py Rust code
├╴at src/error.rs:54:21
│
├─▶ execute query
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-session/src/session.rs:319:14
│
├─▶ internal compute error: spawn compute executor
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute.rs:268:6
│
├─▶ internal compute error: unable to create operation
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute/operation.rs:435:6
│
├─▶ internal compute error: spead failure
│   ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute/operation/merge.rs:171:106
│
╰─▶ Unsupported type Struct([Field { name: "ts", data_type: Timestamp(Nanosecond, None), nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "user", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "text", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "reactions", data_type: List(Field { name: "item", data_type: Struct([Field { name: "count", data_type: Int64, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "name", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "users", data_type: List(Field { name: "item", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }), nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }]), nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }), nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }]) for list spread
    ╰╴at /Users/ryan.michael/work/kaskada/crates/sparrow-runtime/src/execute/operation/merge.rs:171:92
```

In [ ]:
duration = datetime.timedelta(minutes=5)

shifted_coversation = conversations.shift_by(duration)  # !!!

#reaction_users = conversations.col("reactions").col("name") #.collect(kt.windows.Trailing(duration)).flatten()  # !!!
#participating_users = conversations.col("user").collect(kt.windows.Trailing(duration))  # !!!
engaged_users = conversations #kt.union(reaction_users, participating_users)  # !!!

examples = kt.record({ "prompt": shifted_coversation, "completion": engaged_users}) \
    .filter(shifted_coversation.is_not_null())

examples.preview(5)

## Fine-tune a model

### Create training dataset

In [ ]:
from sklearn import preprocessing

examples_df = examples.run().to_pandas()

le = preprocessing.LabelEncoder()
le.fit(examples_df.completion.explode())

# Format for the OpenAI API
def format_prompt(prompt):
    return "start -> " + "\n\n".join([f' {msg["user"]} --> {msg["text"]} ' for msg in prompt]) + "\n\n###\n\n"
examples_df.prompt = examples_df.prompt.apply(format_prompt)

def format_completion(completion):
    return " " + (" ".join([le.transform(u) for u in completion]) if len(completion) > 0 else "nil") + " end"
examples_df.completion = examples_df.completion.apply(format_completion)

# Write examples to file
examples_df.to_json("examples.jsonl", orient='records', lines=True)

### Upload to OpenAI

In [ ]:
from types import SimpleNamespace
from openai import cli

# verifiy data format, split for training & validation
args = SimpleNamespace(file='./examples.jsonl', quiet=True)
cli.FineTune.prepare_data(args)
training_id = cli.FineTune._get_or_upload('./examples_prepared_train.jsonl', True)

### Create the training job

In [ ]:
import openai

resp = openai.FineTune.create(
    training_file = training_id,
    model = "davinci",
    n_epochs = 2,
    learning_rate_multiplier = 0.02,
    suffix = "coversation_users"
)
print(f'Fine-tuning model with job ID: "{resp["id"]}"')